In [1]:
import warnings
import datetime

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from IPython.display import display, Markdown , Math 

sns.set()
warnings.filterwarnings('ignore')

In [2]:
def printmd(string): display(Markdown(string))
def latex(out): printmd(f'{out}')  
def pr(string): printmd('***{}***'.format(string))

<h1>Naive Bayes Classifier</h1>

<h2>
  <p>
    <a href =   "https://github.com/daodavid" > 
         author: daodeiv (David Stankov) 
       <img src="https://cdn.thenewstack.io/media/2014/12/github-octocat.png" align="left" width="120"  alt="daodavid" ></a>
    </p>      
</h2>   

<h2 id=''> Bayes Theorem </h2>
<h6>
  <font size="4" face = "Times New Roma" color='#3f134f' > 
    <ul style="margin-left: 30px">
      <li><a href='#bayes_theorem'>Bayes Theorem</a> </li> <br>
      <!--<li><a href='#int-1'>Introduction </a> </li><br> -->
      <li><a href='#deff_softmax'>Softmaxt definition and  how it works?</a> </li><br>
      <li><a href='#optimization'>Optimizaton of  Softmax Loss with Gradient Descent (Deep math calculation)</a> </li><br>  
      <li><a href='#impl'>Implementation of Softmax using numpy </a> </li><br>
       <li><a href='#reg'>Regularization of softmax by learning rate and max iterations</a> </li><br> 
       <li><a href='#conclusion'>Conclusion</a> </li><br>  
        
</ul>    
 </font>
  </h6>
  

Naive Bayes is one of the simplest supervised ML algorithms meanwhile very efficient and also is able to learn fast and make a quick prediction, therefore it is so useful and popular.
Naive Bayes contains two words Naive and Bayes, Bayes because it is built on Bayes Theorem, and Naive because it assumes that features are independent even if they actually are interdependent.It is simple but very powerful and works well with large datasets and sparse matrices. It works really well on text classification problems, and spam filtering.

<h2 id='bayes_theorem'> Bayes Theorem </h2>

Bayes theorem describes a probability of an event, based on prior knowledge of conditions that might be related to an event.
First, let's take the formula of conditional probability and try to derive Bayes Theorem:

$$p(A|B) = p(B\cap A)/p(B)$$

Probability of event A given B, meaning what is the probability of A when event B is already taken place, which is equal to the probability of A intersection B (the probability of both A and B events are taking place) divided by the probability of B. <br>

we have the same for probability of event B given event A $$p(B|A) = p(A\cap B)/p(A)$$
the  $p(A\cap B)$ and  $p(B\cap A)$ are basicaly the same. Since they are the same, we can get two formulas and move denominator to the left of the equation,and equate them 
$$ p(B|A)p(A) = p(A\cap B) = p(B \cap A) = p(A|B)p(B) $$

So, when we want to find probability of A given B we can write our equation on this way: <br> <br>
$$P(A|B) = P(B|A) * P(A) / P(B)$$,<br> <br> and this is the equation of Bayes Theorem

* P(A|B) is the posterior probability of class (target) given predictor (attribute).
* P(B) is the prior probability of class.
* P(B|A) is the likelihood which is the probability of predictor given class.
* P(A) is the prior probability of predictor.

<h2 id='works'>  How does Binomial Naive Bayes work? </h2>

For our purposes we going to use <a href='https://www.kaggle.com/datasets/priy998/golf-play-dataset'>Golf Play Dataset<a>

In [4]:
iris = pd.read_csv("../../../resources/data/golf_df.csv")
iris

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


We classify whether the day is suitable for playing golf, given the features of the day. The columns represent these features and the rows represent individual entries. If we take the first row of the dataset, we can observe that is not suitable for playing golf if the outlook is rainy, temperature is hot, humidity is high and it is not windy. We make two assumptions here, one as stated above we consider that these predictors are independent. That is, if the temperature is hot, it does not necessarily mean that the humidity is high. Another assumption made here is that all the predictors have an equal effect on the outcome. That is, the day being windy does not have more importance in deciding to play golf or not

According to this example, Bayes theorem can be rewritten as: <br> <br>
$$P(y|X) = P(X|y) * P(y) / P(X)$$ <br> <br>
The variable y is the class variable(play golf), which represents if it is suitable to play golf or not given the conditions. Variable X represent the parameters/features.

X is given as , <br>
 $$X = (x_1,x_2,...,x_n)$$ <br> <br>
 Here x_1,x_2….x_n represent the features, i.e they can be mapped to outlook, temperature, humidity and windy. By substituting for X and expanding using the chain rule we get,

because we assume that features x_i are independent we can write for all feature bayes formula as following:

$$P(y| x_1,x_2,...,x_n ) = \frac{P(x_1|y).P(x_2|y)...P(x_n|y)P(y)}{P(x_1)P(x_2)...P(x_n)} $$

When the predictors take up a continuous value and are not discrete, we assume that these values are sampled from a gaussian distribution.

$$P(x_i|y) = \frac{1}{(2\pi\sigma^2_y)^(1/2)}exp\big(- \frac { (x_i - \mu_y)^2}{2\sigma^2_y}\big) $$

In our data set the variable are descrete.

Now, you can obtain the values for each by looking at the dataset and substitute them into the equation. For all entries in the dataset, the denominator does not change, it remain static. Therefore, the denominator can be removed and a proportionality can be introduced.
$$P(y| x_1,x_2,...,x_n ) \propto p(y)\prod_{i=0}^{n}P(x_i|y)$$

$P(y="yes"|\;x_1="overcast") \propto P(x_1="overcast"|\;y="yes" )P(y="yes")$

In [38]:
df = iris
df.groupby("Outlook")["Play"].count().to_numpy()

array([4, 5, 5], dtype=int64)

In [39]:
label = "Play"
yes = df[df[label] == "yes"].groupby("Outlook")[label].count()
no = df[df[label] == "no"].groupby("Outlook")[label].count()

In [44]:
yes

Outlook
overcast    4
rainy       3
sunny       2
Name: Play, dtype: int64

In [40]:
k = yes/yes.sum()
yes
k

Outlook
overcast    0.444444
rainy       0.333333
sunny       0.222222
Name: Play, dtype: float64

In [41]:
k.index = [(lambda i: f'P(x= "{i}"|y="yes") =')(i) for i in k.index] 

In [42]:
k

P(x= "overcast"|y="yes") =    0.444444
P(x= "rainy"|y="yes") =       0.333333
P(x= "sunny"|y="yes") =       0.222222
Name: Play, dtype: float64

In [43]:
no = df[df[label] == "no"].groupby("Outlook")[label].count()
k = no/no.sum()
k.index = [(lambda i: f'P(x= "{i}"|y="yes") =')(i) for i in k.index] 
k

P(x= "rainy"|y="yes") =    0.4
P(x= "sunny"|y="yes") =    0.6
Name: Play, dtype: float64

if we get P(x= "sunny"|y="No") =    0.6 <br>
and P(x= "sunny"|y="yes") =       0.222222 this mean if we get only synny ito acount we will conclude that y= no

<h2> References </h2>
* <a href='https://towardsdatascience.com/implementing-naive-bayes-algorithm-from-scratch-python-c6880cfc9c41'>naive <br>
* <a href='https://prwatech.in/blog/machine-learning/naive-bayes-classifier-in-machine-learning/'> Indian Naive Bayes </a> <br>
* <a href='https://www.geeksforgeeks.org/naive-bayes-classifiers/'>Naive Bayes Classifiers </a>    
    * <a href='https://towardsdatascience.com/naive-bayes-classifier-81d512f50a7c'>Naive Bayes Classifiers </a>  

Now, you can obtain the values for each by looking at the dataset and substitute them into the equation. For all entries in the dataset, the denominator does not change, it remain static. Therefore, the denominator can be removed and a proportionality can be introduced.
$$P(y| x_1,x_2,...,x_n ) \propto p(Y)\prod_{i=0}^{n}P(x_i|y)$$
